# Using Principal Component Analysis for Digital Signal Processing
### Ayush Chakraborty, Allyson Hur, Cherry Pham, Suki Sacks

## Abstract
Digital Signal Processing (DSP) involves manipulating and analyzing signals to extract meaningful information. DSP normally relies on predefined algorithms and mathematical models for signal-processing tasks. This project explores the integration of principal component analysis (PCA) into DSP to enhance its capabilities and adaptability.

The objective of this project is to investigate how PCA can be used for tasks such as signal denoising, modulation recognition, and feature extraction. By utilizing datasets containing diverse signal patterns, the project aims to develop a deeper understanding of the adaptability and versatility of PCA in regards to various signal processing challenges.


## Implementation

Imports

In [ ]:
from itertools import islice
import scipy.stats as stats
import numpy as np

Sampling Parameters

In [ ]:
f_sample_hz = 1e6
dt = 1 / f_sample_hz
samples_per_symbol = 200
T_symbol = samples_per_symbol * dt

symbol_zeros = np.zeros((samples_per_symbol,))

# Zero-mean Gaussian noise.
variance = 1e-3
noiseRV = stats.norm(loc=0.0, scale=np.sqrt(variance))

Batch Data

In [ ]:
def batched(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch

Create a binary message

In [ ]:
def generate_signal(message_txt: str):
    message_bits_txt = "".join([f"{ord(x):08b}" for x in message_txt])
    print(f"T: {message_txt} -> {message_bits_txt}")
    message_bits = np.array([int(b) for b in message_bits_txt])

    return message_bits, message_bits_txt


message_txt = "Hello, world."
message_bits, message_bits_txt = generate_signal(message_txt)

### **Transmitter Side**
m[k] -> Encoder -> Symbols -> Modulator -> x(t)


### Symbol Construction

In [ ]:
def PAMGenerator(symbol, k=2, samples_per_symbol=10):
    amplitudes = np.linspace(-1, 1, k)
    if symbol < 0 or symbol > (k - 1):
        raise ValueError(f"{k}-PAM must have symbols in [0, {k-1}].")
    return amplitudes[symbol] * np.ones(
        samples_per_symbol,
    )
    
message_txt = "Hello, world."
message_bits, message_bits_txt = generate_signal(message_txt)

In [ ]:
def generate_symbols(bits_per_symbol: int):
    symbols_of_t = []
    symbol_zeros = np.zeros((samples_per_symbol,))
    N_symbols = 2**bits_per_symbol
    for symbol in range(N_symbols):
        x = PAMGenerator(symbol, k=N_symbols, samples_per_symbol=samples_per_symbol)
        symbols_of_t.append(x)

        # Note that we are concatenating our waveform with zeros before and after -
        # this will make the plot and FFT a little more obvious.
        x = np.concatenate((symbol_zeros, x, symbol_zeros))

    return symbols_of_t
bits_per_symbol = 2
symbols_of_t = generate_symbols(bits_per_symbol)

### Encoding

### Modulation

### **Channel Model**

### **Receiver Side**
y(t) -> IQ Demodulator -> Symbols -> Decoder -> m_hat[k]